# Segmentation metric tutorial
Code and tutorial of the calculating MaskRCNN segmentation metric 

In [78]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [91]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import tensorflow as tf
import metric as met

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"

# Root directory of the project
ROOT_DIR = os.path.abspath("../../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library
from mrcnn import utils
from mrcnn import visualize
from mrcnn.visualize import display_images, random_colors
import mrcnn.model as modellib
from mrcnn.model import log

from samples.iris import iris

%matplotlib inline 

# Directory to save logs and trained model
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

# Path to Ballon trained weights
# You can download this file from the Releases page
# https://github.com/matterport/Mask_RCNN/releases
IRIS_WEIGHTS_PATH = "../../logs/iris20190416T0230/mask_rcnn_iris_0030.h5"  # TODO: update this path

# Configurations

In [80]:
config = iris.IrisConfig()
IRIS_DIR = os.path.join(ROOT_DIR, "datasets/iris534")

In [81]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  1280
IMAGE_META_SIZE                18
IMAGE_MIN_DIM                  960
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [1280 1280    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'mrcnn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'rpn_class_loss': 1.0, 'mrcnn_mask_loss': 1.0, 'mrcnn_bbox_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE         

# Notebook Preference

In [82]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
DEVICE = "/gpu:0"  # /cpu:0 or /gpu:0

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"

In [83]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

In [84]:
def get_colors(class_ids, color_palate):
    color_list = []
    
    for i, instance_id in enumerate(class_ids):
        color_list.append(color_palate[instance_id])
    return color_list

# Load Validation Dataset

In [85]:
# Load validation dataset
dataset = iris.IrisDataset()
dataset.load_iris(IRIS_DIR, "val")

# Must call before using the datset
dataset.prepare()

print("Images: {}\nClasses: {}".format(len(dataset.image_ids), dataset.class_names))

Images: 54
Classes: ['BG', 'iris', 'pupil', 'light_source', 'hole', 'anw']


# Load Model

In [86]:
# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode="inference", model_dir=MODEL_DIR,
                              config=config)

In [87]:
# Load the last model you trained
#weights_path = model.find_last()
weights_path = IRIS_WEIGHTS_PATH

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

Loading weights  ../../logs/iris20190416T0230/mask_rcnn_iris_0030.h5
Re-starting from epoch 30


In [88]:
dataset.image_ids

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50,
       51, 52, 53])

# Metric - Image load & prepare input data

In [89]:
label_list = dataset.class_names
target_predict_list = []

for image_id in tqdm(dataset.image_ids):
    
    image, image_meta, gt_class_id, gt_bbox, gt_mask =\
            modellib.load_image_gt(dataset, config, image_id, use_mini_mask=False)

    results = model.detect([image], verbose=1)
    r = results[0]
    
    npad = ((160,160),(0,0),(0,0))

    target_mask, target_fit_list = dataset.load_mask(image_id) 
    target_mask = np.pad(target_mask, npad, 'constant', constant_values=("0"))
    target = [target_fit_list, target_mask]

    pred_fit_list = r['class_ids']
    pred_mask = r['masks']
    predict = [pred_fit_list, pred_mask]
    
    target_predict_list += [[target,predict]]

  0%|          | 0/54 [00:00<?, ?it/s]

Processing 1 images
image                    shape: (1280, 1280, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1280, 1280, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 18)               min:    0.00000  max: 1280.00000  int64
anchors                  shape: (1, 409200, 4)        min:   -0.28306  max:    1.23302  float32


  2%|▏         | 1/54 [00:52<46:38, 52.80s/it]

Processing 1 images
image                    shape: (1280, 1280, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1280, 1280, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 18)               min:    0.00000  max: 1280.00000  int64
anchors                  shape: (1, 409200, 4)        min:   -0.28306  max:    1.23302  float32


  4%|▎         | 2/54 [00:54<32:29, 37.49s/it]

...

 96%|█████████▋| 52/54 [02:44<00:04,  2.05s/it]

Processing 1 images
image                    shape: (1280, 1280, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1280, 1280, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 18)               min:    0.00000  max: 1280.00000  int64
anchors                  shape: (1, 409200, 4)        min:   -0.28306  max:    1.23302  float32


 98%|█████████▊| 53/54 [02:45<00:01,  1.95s/it]

Processing 1 images
image                    shape: (1280, 1280, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 1280, 1280, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 18)               min:    0.00000  max: 1280.00000  int64
anchors                  shape: (1, 409200, 4)        min:   -0.28306  max:    1.23302  float32


100%|██████████| 54/54 [02:47<00:00,  1.78s/it]


# Metric - sementic segmentation metric

In [92]:
metric_result = met.sementic_seg_metric(target_predict_list,label_list)
print(metric_result)

100%|██████████| 54/54 [00:16<00:00,  5.35it/s]

{'anw': 0.8507169178950528, 'iris': 0.9217449863301803, 'pupil': 0.9215811914262116, 'light_source': 0.4265750131109166, 'hole': 0.24523697588107365}


# Metric - instance segmentation metric

In [ ]:
metric_result = met.mask_rcnn_metric(target_predict_list,label_list)
print(metric_result)

  0%|          | 0/10 [00:00<?, ?it/s]